In [ ]:
"""
Created on Mon Mar 22 10:27 2021

This script is to prepare the thermal forcing 2D fields to be used for tuning the simple parametrizations
with dask

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.melt_functions as meltf
from basal_melt_param.constants import *
import basal_melt_param.T_S_profile_functions as tspf
from scipy import stats
from dask import delayed
import pickle
import matplotlib as mpl

import distributed
import glob

In [ ]:
%matplotlib qt5

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

READ IN DATA

In [ ]:
nemo_run = 'isfru94'
HUB = False

In [ ]:
if nemo_run in ['ctrl94','isf94','isfru94']:
    inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/NEMO_'+nemo_run+'_ANT_STEREO/'
    inputpath_mask = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
    outputpath_simple = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
    inputpath_plumes = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    outputpath = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/'
else:
    inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
    inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
    inputpath_profiles = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/T_S_PROF/nemo_5km_'+nemo_run+'/'
    outputpath_simple = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_'+nemo_run+'/'
    inputpath_plumes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/PLUMES/nemo_5km_'+nemo_run+'/'
    outputpath = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/'

inputpath_hub='/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/HUB_from_Garrett/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

chunk_size = 300

Files for param

In [ ]:
file_other = xr.open_mfdataset(inputpath_data+'corrected_draft_bathy_isf.nc')
file_other_cut = uf.cut_domain_stereo(file_other, map_lim, map_lim).chunk(chunks={'x': chunk_size, 'y': chunk_size})
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_bed_cut = uf.cut_domain_stereo(file_mask_orig['bathy_metry'], map_lim, map_lim).chunk(chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
#file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc') # separate FRIS
if nemo_run in ['ctrl94','isf94','isfru94']:
    file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS.nc')
else:
    file_isf_orig = xr.open_mfdataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_Nisf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_Nisf).chunk(chunks={'x': chunk_size, 'y': chunk_size})
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
if nemo_run in ['ctrl94','isf94','isfru94']:
    file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_allyy.nc', chunks={'Nisf': 1})
else:
    file_TS_orig = xr.open_mfdataset(inputpath_profiles+'T_S_mean_prof_corrected_km_contshelf_and_offshore_1980-2018_oneFRIS.nc', chunks={'Nisf': 1})

file_TS = file_TS_orig.sel(Nisf=large_Nisf)

In [ ]:
param_var_of_int = file_isf[['ISF_mask', 'latitude', 'longitude', 'front_bot_depth_max', 'front_bot_depth_avg']]
#plume_charac = xr.open_mfdataset(inputpath_plumes+'nemo_5km_plume_characteristics.nc', chunks={'x': chunk_size, 'y': chunk_size})

In [ ]:
ice_draft_pos = file_other_cut['corrected_isfdraft']
ice_draft_neg = -ice_draft_pos

In [ ]:
if HUB:
    with open(inputpath_hub + "nemoHUB_"+nemo_run+".pickle","rb") as f:
         [hub_by_shelf,hub_full] = pickle.load(f)
    hub_da = file_isf['ISF_mask']*0 + hub_full
    hub_da = -1 * hub_da.where(file_isf['ISF_mask'] > 1) 
    hub_da = hub_da.rename('HUB_depth')

First 2D fields of thermal forcing

In [ ]:
# find temperature and salinity at the given depth
filled_TS = file_TS.ffill(dim='depth')

In [ ]:
bottom = False
if bottom:
    depth_of_int = param_var_of_int['front_bot_depth_avg']
    ice_draft_isf = ice_draft_pos.where(param_var_of_int['ISF_mask'] == file_isf.Nisf).chunk(chunks={'Nisf': 1})
    T0 = filled_TS['theta_ocean'].interp(depth=depth_of_int).chunk(chunks={'Nisf':1}).where(file_isf_mask==file_isf.Nisf).chunk(chunks={'Nisf':1, 'x': chunk_size, 'y': chunk_size})
    S0 = filled_TS['salinity_ocean'].interp(depth=depth_of_int).chunk(chunks={'Nisf':1}).where(file_isf_mask==file_isf.Nisf).chunk(chunks={'Nisf':1, 'x': chunk_size, 'y': chunk_size})
elif HUB:
    ice_draft_isf = ice_draft_pos.where(param_var_of_int['ISF_mask'] == file_isf.Nisf).chunk(chunks={'Nisf': 1})
    depth_of_int_all = ice_draft_pos.where(ice_draft_pos<hub_da, hub_da) # ice draft depth or HUB depth
    depth_of_int = depth_of_int_all.where(param_var_of_int['ISF_mask'] == file_isf.Nisf,0).chunk(chunks={'Nisf': 1})
    T0 = filled_TS['theta_ocean'].interp(depth=depth_of_int)
    S0 = filled_TS['salinity_ocean'].interp(depth=depth_of_int)
else:
    ice_draft_isf = ice_draft_pos.where(param_var_of_int['ISF_mask'] == file_isf.Nisf).chunk(chunks={'Nisf': 1})
    depth_of_int = ice_draft_isf.where(ice_draft_isf<param_var_of_int['front_bot_depth_max'], 
                                   param_var_of_int['front_bot_depth_max']).chunk(chunks={'Nisf':1}) # ice draft depth or deepest entrance depth
    #T0 = (file_TS['mean_theta']-0.5).interp(depth=depth_of_int)
    #S0 = file_TS['mean_salinity'].interp(depth=depth_of_int)
    T0 = filled_TS['theta_ocean'].interp(depth=depth_of_int)
    S0 = filled_TS['salinity_ocean'].interp(depth=depth_of_int)

Tf = meltf.freezing_temperature(S0, -ice_draft_isf)

In [ ]:
# thermal forcing term
#thermal_forcing =  T0 - Tf
thermal_forcing = T0 - Tf # trying to correct bias of Ross and Ronne

# project it on 2D
thermal_forcing_1map = thermal_forcing.sum('Nisf').where(file_isf_mask>1)

In [ ]:
#thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_for_tuning.nc','w')
#thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_for_tuning_only_contshelf.nc','w')
if bottom:
    #thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected.nc','w')
    thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_bottom_for_tuning_corrected_oneFRIS.nc','w')
elif HUB:
    thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_HUB_for_tuning_corrected_oneFRIS.nc','w')
else:
    #thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected.nc','w')
    thermal_forcing_1map.to_dataset(name='thermal_forcing_2D').to_netcdf(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc','w')

Prepare additionally S0 for new formulation

In [ ]:
S0_clean = S0.where(file_isf['ISF_mask']==depth_of_int.Nisf)
S_test = S0_clean.sum('Nisf').where(file_isf_mask>0)

In [ ]:
if bottom:
    #S_test.to_dataset(name='salinity_depth_of_int').to_netcdf(outputpath_simple+'input_salinity_2D_bottom_for_tuning_corrected.nc','w')
    S_test.to_dataset(name='salinity_depth_of_int').to_netcdf(outputpath_simple+'input_salinity_2D_bottom_for_tuning_corrected_oneFRIS.nc','w')
elif HUB:
    S_test.to_dataset(name='salinity_depth_of_int').to_netcdf(outputpath_simple+'input_salinity_2D_HUB_for_tuning_corrected_oneFRIS.nc','w')
else:
    #S_test.to_dataset(name='salinity_depth_of_int').to_netcdf(outputpath_simple+'input_salinity_2D_for_tuning_corrected.nc','w')
    S_test.to_dataset(name='salinity_depth_of_int').to_netcdf(outputpath_simple+'input_salinity_2D_for_tuning_corrected_oneFRIS.nc','w')

In [ ]:
#TS_orig = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected.nc')
TS_orig = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_corrected_oneFRIS.nc')
#TS_minus_0point5 = xr.open_dataset(outputpath_simple+'thermal_forcing_2D_for_tuning_minus0-5.nc')

In [ ]:
file_isf['isf_area_here'].sel(Nisf=11).load()

In [ ]:
nisf_list = file_isf.Nisf.values.tolist()
nisf_list.remove(11)
nisf_list.remove(10)

In [ ]:
file_isf['isf_name'].sel(Nisf=31).load()

In [ ]:
TS_orig.where(file_isf['ISF_mask']==75,drop=True).min().load()

In [ ]:
(TS_orig['thermal_forcing_2D'].isel(profile_domain=0, time=10) - TS_minus_0point5['thermal_forcing_2D'].isel(profile_domain=0, time=10)).plot()

In [ ]:
plt.close('all')